# Ejercicio 1

## Creación de una bases de datos de películas extraídas de un API

En este ejercicio vamos a poner en práctica lo que hemos aprendido sobre la extracción de contenido de APIs y la creación de una base de datos con sus tablas. Siguiendo las fases propuestas, conseguiremos extraer datos de películas, almacenarlos en un DataFrame y finalmente crear una base de datos con la estructura adecuada.

Fase 1: Extracción de datos de películas

Nuestro primer objetivo es extraer los datos de películas de la API. En Adalab nos hemos creado un API
muy sencilla que contiene la siguiente información:
Datos a extraer:
- Título
- Año de lanzamiento
- Duración (en minutos)
- Género
- Contenido para adultos (sí o no)


El objetivo es extraer 100 películas de esta API utilizando el siguiente endpoint:
https://beta.adalab.es/resources/apis/pelis/pelis.json

In [15]:
import requests
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error

In [16]:
url_films = "https://beta.adalab.es/resources/apis/pelis/pelis.json"
films = requests.get(url_films)

In [17]:
films.status_code

200

In [18]:
film_data = films.json()
film_data

[{'id': 1,
  'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 2,
  'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 3,
  'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 4,
  'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False,
  'subtitulos': ['es', 'en', 'fr']},
 {'id': 5,
  'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 6,
  'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 7,
  'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos

In [19]:
df_film_data = pd.DataFrame(film_data)
df_film_data

,id,titulo,año,duracion,genero,adultos,subtitulos
0,1,The Godfather,1972,175,Crimen,False,"[es, en]"
1,2,The Godfather Part II,1974,202,Crimen,False,"[es, en]"
2,3,Pulp Fiction,1994,154,Crimen,True,"[es, en]"
3,4,Forrest Gump,1994,142,Drama,False,"[es, en, fr]"
4,5,The Dark Knight,2008,152,Acción,False,"[es, en]"
...,...,...,...,...,...,...,...
95,96,La vita è bella,1997,116,Drama,False,"[es, en, it]"
96,97,Requiem for a Dream,2000,102,Drama,True,"[es, en]"
97,98,Memento,2000,113,Thriller,True,"[es, en]"
98,99,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False,"[es, en]"


# Inserción de los Datos en la Base de Datos

En este apartado vamos a realizar la inserción de los datos extraídos en las tablas creadas en nuestra base de datos MySQL.

In [38]:
try:
    cnx = mysql.connector.connect(
        host='localhost',
        user='root',
        password='AlumnaAdalab',
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS films_db"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Conexión exitosa
Query exitosa


In [21]:
mycursor.execute("USE films_db")

query = '''CREATE TABLE IF NOT EXISTS films(
	idFilm INT,
    title VARCHAR(70) NOT NULL,
    year YEAR,
    runtime INT,
    genre VARCHAR(45),
    rating BOOLEAN,
    PRIMARY KEY (idFilm)
);'''
mycursor.execute(query)

query = '''CREATE TABLE IF NOT EXISTS languages(
    idLang INT AUTO_INCREMENT,
    language VARCHAR(45) UNIQUE NOT NULL,
    PRIMARY KEY (idLang)
);'''
mycursor.execute(query)

query = '''CREATE TABLE IF NOT EXISTS subtitles_films(
    idFilm INT NOT NULL,
    idLang INT NOT NULL,
    PRIMARY KEY (idFilm, idLang),
    FOREIGN KEY (idFilm) REFERENCES films(idFilm),
    FOREIGN KEY (idLang) REFERENCES languages(idLang)
);'''

mycursor.execute(query)

In [22]:
mycursor.execute("USE films_db")

for index, film in df_film_data.iterrows(): 
    
    query_insert = '''INSERT INTO films (idFilm, title, year, runtime, genre, rating) 
                      VALUES (%s, %s, %s, %s, %s, %s);
                      '''
    
    values = (
        film['id'], 
        film['titulo'], 
        film['año'], 
        film['duracion'], 
        film['genero'], 
        film['adultos']
    )
    
    mycursor.execute(query_insert, values)

    idFilm = film['id']

    for subtitle in film['subtitulos']:       
        
        idLang = None
        query_insert_lang = '''INSERT IGNORE INTO languages (language) 
                            VALUES (%s);
                            '''
        # con el IGNORE aunque se ignore el valor que se repite, el autoincrement reserva un ID para ese valor ignorado.
        mycursor.execute(query_insert_lang, (subtitle,))

        query_select_lang = """SELECT idLang
                            FROM languages
                            WHERE language = %s
                            """
        mycursor.execute(query_select_lang, (subtitle,))        
        
        idLang = mycursor.fetchone()[0]

        query_insert_relation = '''INSERT INTO subtitles_films (idFilm, idLang) 
                                VALUES (%s, %s)'''
        
        valuesLang = (
            idFilm, 
            idLang
        )
        
        mycursor.execute(query_insert_relation, valuesLang)


cnx.commit()
cnx.close()




# df_clean = df_film_data.replace({np.nan: None, 'nan': None, 'NaN': None})
# data = df_clean.values.tolist()
# mycursor.executemany(query_insert, data)
# cnx.commit()

IntegrityError: 1062 (23000): Duplicate entry '1' for key 'films.PRIMARY'

¿Cuántas películas tienen una duración superior a 120 minutos?

In [39]:
mycursor.execute("USE films_db")

query = '''SELECT COUNT(idFilm) as CantidadPeliculas 
        FROM films
        WHERE runtime >= 120'''
mycursor.execute(query)
df = pd.read_sql(query, cnx)
print(df)

C:\Users\User\AppData\Local\Temp\ipykernel_15388\1560585611.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnx)


InternalError: Unread result found

¿Cuántas películas incluyen subtítulos en español?

¿Cuántas películas tienen contenido adulto?

¿Cuál es la película más antigua registrada en la base de datos?

Muestra el promedio de duración de las películas agrupado por género.

¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.

¿Cuál es el año con más películas en la base de datos

Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

Muestra todas las películas cuyo título contenga la palabra "Godfather" (puedes usar cualquier palabra).